# 第四章：迭代器与生成器

In [1]:
#4.1 手动遍历迭代器
#问题：你想遍历一个可迭代对象中的所有元素，但是却不想使用for循环
#解决方案：为了手动遍历可迭代对象，使用next函数并在代码中捕获STOPITERATION异常，比如

In [8]:
def manual_iter():
    with open('somefile.txt') as f:
        try:
            while True:
                line = next(f)
                print(line, end='')
        except StopIteration:
            pass

In [10]:
help(next)

Help on built-in function next in module builtins:

next(...)
    next(iterator[, default])
    
    Return the next item from the iterator. If default is given and the iterator
    is exhausted, it is returned instead of raising StopIteration.



In [9]:
with open('somefile.txt') as f:
    while True:
        line = next(f)
        if line is None:
            break
        print(line, end='')

=== Keeping the Last N Items

==== Problem

You want to keep a limited history of the last few items seen
during iteration or during some other kind of processing.

==== Solution

Keeping a limited history is a perfect use for a `collections.deque`.
For example, the following code performs a simple text match on a
sequence of lines and prints the matching line along with the previous
N lines of context when found:

[source,python]
----
from collections import deque

def search(lines, pattern, history=5):
    previous_lines = deque(maxlen=history)
    for line in lines:
        if pattern in line:
            for pline in previous_lines:
                print(lline, end='')
            print(line, end='')
            print()
        previous_lines.append(line)

# Example use on a file
if __name__ == '__main__':
    with open('somefile.txt') as f:
         search(f, 'python', 5)
----

==== Discussion

Using `deque(maxlen=N)` creates a fixed size queue.  When new items
are added and the q

StopIteration: 

大多数情况下，我们会使用for循环语句用来遍历一个可迭代对象，但是，偶尔也需要对迭代做
更加精准的控制，这时候理解底层迭代机制就显得尤为重要了

In [12]:
items = [1, 2, 3]
# Get the iterator
it = iter(items)
# run the iterator
next(it), next(it), next(it)

(1, 2, 3)

In [13]:
next(it)

StopIteration: 

## 4.2 代理迭代

问题：你构建了一个自定义容器，里面包含有列表，元组和其他可迭代对象，你想直接在这个
新容器上执行迭代操作

In [14]:
#解决方案：实际上你有只需要一个__iter__()方法，讲迭代操作代理容器内部的对象上去

In [ ]:
class Node:
    def __init__(self, value):
        self._value = value
        self.children = []
        
    def __repr__(self):
        return 'Node({!r})'.format(self._value) # 将对象格式化转换成repr格式，计算机能看懂的格式
    
    def add_child(self, node):
        self._children.append(node)
        
    def __iter__(self):
        return iter(self._children)
    
if __name__ == "__main__":
    root = Node(0)
    child1 = Node(1)
    child2 = Node(2)
    root.add_child(child1)
    root.add_child(child2)
    # Outputs Node(1), Node(2)
    for ch in root:
        print(ch)
#__iter__方法只是简单的将迭代请求？传递给内部的_children属性
#__repr__,__iter__是怎么起作用的

In [17]:
print(repr('hello')), print('hello')

'hello'
hello


(None, None)

In [22]:
#Python的迭代器协议需要__iter__方法返回一个实现了__next__方法的迭代器对象，如果你只是#迭代请求
#遍历其他容器的内容，你无需担心底层是怎样实现的，你所要做的只是传递迭代请求即可

In [21]:
#这里的iter函数的使用简化了代码，iter只是简单的通过调用s.__iter__()方法来返回对应的迭代器对象
#就跟len(s)会调用s.__len()__原理是一样的

## 4.3 使用生成器创建新的迭代模式

问题：你想实现一个自定义迭代模式，跟普通的内置函数range，reversed函数不一样

解决方案：如果你想实现一种新的迭代模式，使用一个生成器函数来定义它
下面是一个生产某个范围内浮点数的生成器

In [26]:
def frange(start, stop, increment):
    x = start 
    while x < stop:
        yield x
        x += increment
#为了使用这个函数，你可以用for循环迭代它或者使用其他接受一个可迭代对象的函数，例如，
#sum， list等

In [24]:
for n in frange(0,5,0.5):
    print(n)

0
0.5
1.0
1.5
2.0
2.5
3.0
3.5
4.0
4.5


In [25]:
list(frange(0, 1, 0.123))

[0, 0.123, 0.246, 0.369, 0.492, 0.615, 0.738, 0.861, 0.984]

一个函数中需要有一个yield语句即可将其转换为一个生成器，跟普通函数不同的是，生成器
只能用于迭代操作，下面是一个实验，向你展示函数底层的工作机制

In [27]:
def countdown(n):
    print('Starting to count from', n)
    while n > 0:
        yield n
        n -= 1
    print("Done!!!")

In [28]:
c = countdown(5)

In [29]:
c

<generator object countdown at 0x00000198CCA8EB88>

In [30]:
next(c)

Starting to count from 5


5

In [31]:
next(c),next(c)

(4, 3)

In [32]:
next(c), next(c)

(2, 1)

In [33]:
next(c)

Done!!!


StopIteration: 

一个生成器函数主要特征是它只会回应在迭代中使用到的next操作，一旦生成器函数返回退出，
迭代终止，我们在迭代中通常使用for语句会自动处理这些细节，所以无需担心

4.4 实现迭代器协议

问题：你想构建一个能支持迭代操作的自定义对象，并希望找到一个能实现迭代协议的简单方法

解决方案：目前为止，在一个对象上实现迭代最简单的方式是使用一个生成器函数，在4.2小节中
使用Node类来表示属性数据结构，你可能想实现一个以深度优先方式遍历树形节点的生成器

In [35]:
class Node:
    def __init__(self, value):
        self._value = value
        self._children = []
        
    def __repr__(self):
        return 'Node({!r})'.format(self._value)
    
    def add_child(self, node):
        self._children.append(node)
        
    def __iter__(self):
        return iter(self._children)
    
    def depth_first(self):
        yield self
        for c in self:
            yield from c.depth_first() # ?
            
if __name__ == "__main__":
    root = Node(0)
    child1= Node(1)
    child2 = Node(2)
    root.add_child(child1)
    root.add_child(child2)
    child1.add_child(Node(3))
    child1.add_child(Node(4))
    child2.add_child(Node(5))
    
    for ch in root.depth_first():
        print(ch)

#depth first方法简单直观，它首先返回自己本身并迭代每一个子节点并通过调用子节点的depth first
#方法使用yield from语句返回对应元素

Node(0)
Node(1)
Node(3)
Node(4)
Node(2)
Node(5)


In [36]:
#python的迭代协议要求一个__iter__方法返回一个特殊的迭代器对象，这个迭代器对象实现__next__方法
#并通过StopIteration异常表示迭代完成，但是实现这些通常会比较繁琐，下面使用一个关联迭代器
#重新实现

In [39]:
class Node2:
    def __init__(self, value):
        self._value = value
        self._children = []
        
    def __repr__(self):
        return 'Node({!r})'.format(selfl._value)
    
    def add_child(self, node):
        self._children.append(node)
        
    def __iter__(self):
        return iter(self,_children)
    
    def depth_first(self):
        return DepthFirstIterator(self)
    
class DepthFirstIterator(object):
    """
    """
    def __init__(self, start_node):
        self._node = start_node
        self._children_iter = None
        self._child_iter = None
        
    def __iter__(self):
        return self
    
    def __next__(self):
        # Return myself if just started; create an iterator for children
        if self._children_iter is None:
            self._children_iter = iter(self._node)
            return self._node
        #If processing a child, return its next item
        elif self._children_iter:
            try:
                nextchild = next(self._children_iter)
                return nextchild
            except StopIteration:
                self._children_iter = None
                return next(self)
        #advance to the next child and start its iteration
        else:
            self._children_iter = next(self._children_iter).depth_first()
            return next(self)
#DepthFirstIterator类和上面使用的生成器的版本工作原理类似，但是写起来繁琐
#因为迭代器必须在迭代处理过程中维护大量的状态信息，代码晦涩难懂，讲你的迭代器定义
#为一个生成器后一切迎刃而解

## 4.5 反向迭代

问题：你想反方向迭代一个序列

解决方案：使用内置的reversed函数

In [41]:
a = list('12345')
for x in reversed(a):
    print(x)

5
4
3
2
1


In [43]:
#反向迭代仅仅当对象的大小可预先确定或者对象实现__reversed__的特殊方法时才能生效，如果
#两者都不符合，那你必须先将对象转换为一个列表才行
#！如果可迭代对象元素很多的话，将其预先转换为一个列表要消耗大量的内存

In [50]:
#可以在定义类上实现__reversed__()方法来实现反向迭代

class Countdown:
    def __init__(self, start):
        self.start = start
        
    def __iter__(self):
        n = self.start
        while n > 0:
            yield n
            n -= 1
            
    # Reversed iterator
    def __reversed__(self):
        n = 1
        while n <= self.start:
            yield n 
            n += 1

for rr in Countdown(10):
    print(rr)
    
for rr in reversed(Countdown(10)):
    print(rr)

#定义一个反向迭代器可以使得代码非常的高效，不需要先将数据填充到一个列表中然后再去
#反向迭代这个列表

10
9
8
7
6
5
4
3
2
1
1
2
3
4
5
6
7
8
9
10


4.6 带有外部状态的生成器函数

问题：你想顶一个生成器函数，但是它会调用某个你想暴露给用户使用的外部状态值

解决方案：如果你想让你的生成器暴露外部状态给用户，别忘了你可以简单的将它实现一个类
然后把生成器函数放到iter方法中获取，比如

In [67]:
from collections import deque

class linehistory:
    def __init__(self, lines, histlen=3):
        self.lines = lines
        self.history = deque(maxlen=histlen)
        
    def __iter__(self):
        for lineno, line in enumerate(self.lines, 1):
            self.history.append((lineno, line))
            yield line
            
    def clear(self):
        self.history.clear()
        
#为了使用这个类，你可以将它当做一个普通的生成器函数，然而，由于可以创建一个实例对象
#于是你可以访问内部属性值，比如，history属性或者clear方法，代码示例如下

In [68]:
with open('somefile.txt') as f:
    lines = linehistory(f)
    for line in lines:
        if 'python' in line:
            for lineno, hline in lines.history:
                print('{}:{}'.format(lineno, hline), end='')

13:N lines of context when found:
14:
15:[source,python]
30:if __name__ == '__main__':
31:    with open('somefile.txt') as f:
32:         search(f, 'python', 5)


In [69]:
#关于生成器，很容易掉进函数无所不能的陷阱。 
#如果生成器函数需要跟你的程序其他部 分打交道的话(比如暴露属性值，允许通过方法调用来控制等等)
#可能会导致你的代码异常的复杂。 如果是这种情况的话，可以考虑使用上面介绍的定义类的方式。 
#在 __iter__() 方法中定义你的生成器不会改变你任何的算法逻辑。 
#由于它是类的一部分， 所以允许你定义各种属性和方法来供用户使用。 
#一个需要注意的小地方是，如果你在迭代操作时不使用for循环语句，那么你得先调用 iter() 函数。
#比如：

In [70]:
f = open('somefile.txt')
lines = linehistory(f)
next(lines)

TypeError: 'linehistory' object is not an iterator

In [71]:
it = iter(lines)
next(it), next(it)

('=== Keeping the Last N Items\n', '\n')

## 4.7 迭代器切片

问题：你想得到一个由迭代器生成的切片对象，但是标准切片操作并不能做到

解决方案：itertoois.islice()正好适用于在迭代器和生成器上做切片操作，比如

In [58]:
def count(n):
    while True:
        yield n
        n += 1
        
c = count(0)
c[10:20]

TypeError: 'generator' object is not subscriptable

In [73]:
import itertools

for x in itertools.islice(c, 10, 20):
    print(x)

10
11
12
13
14
15
16
17
18
19


In [74]:
#迭代器和生成器不能使用标准的切片操作，因为它们的长度事先我们并不知道(并且也没 有实现索引)。 
#函数 islice() 返回一个可以生成指定元素的迭代器，它通过遍历并丢弃 直到切片开始索引位置的所有元素。 
#然后才开始一个个的返回元素，并直到切片结束索 引位置。 
#这里要着重强调的一点是 islice() 会消耗掉传入的迭代器中的数据。 必须考虑到迭代器 是不可逆的这个事实。 
#所以如果你需要之后再次访问这个迭代器的话，那你就得先将它 里面的数据放入一个列表中

4.8 跳过可迭代对象的开始部分

问题：你想遍历一个可迭代对象，但是它开始某些元素你并感兴趣，跳过他们

解决方案：itertools模块中有一些函数可以完成这个任务，首先介绍的是
itertools.dropwhile函数，使用时，你给它传递一个函数对象和一个可迭代对象
它会返回一个迭代器对象，丢弃原有序列中直到函数返回True之前的所有元素
，然后返回后面所有元素

In [3]:
#假定你再读取一个开始部分是几行注释的源文件

with open('somefile.txt') as f:
    for line in f:
        print(line, end='')

### Keeping the Last N Items

#### Problem

You want to keep a limited history of the last few items seen
during iteration or during some other kind of processing.

#### Solution

Keeping a limited history is a perfect use for a `collections.deque`.
For example, the following code performs a simple text match on a
sequence of lines and prints the matching line along with the previous
N lines of context when found:

[source,python]
----
from collections import deque

def search(lines, pattern, history#5):
    previous_lines # deque(maxlen#history)
    for line in lines:
        if pattern in line:
            for pline in previous_lines:
                print(lline, end#'')
            print(line, end#'')
            print()
        previous_lines.append(line)

# Example use on a file
if __name__ ## '__main__':
    with open('somefile.txt') as f:
         search(f, 'python', 5)
----

#### Discussion

In [7]:
#跳过开始部分的注释行的话

from itertools import dropwhile

with open('somefile.txt') as f:
    for line in dropwhile(lambda line: line.startswith('#'), f):
        print(line, end='')

##
#
##此种情况下跳不过，会从这开始显示

You want to keep a limited history of the last few items seen
during iteration or during some other kind of processing.
#### Solution
Keeping a limited history is a perfect use for a `collections.deque`.
For example, the following code performs a simple text match on a
sequence of lines and prints the matching line along with the previous
N lines of context when found:

[source,python]
----
from collections import deque

def search(lines, pattern, history#5):
    previous_lines # deque(maxlen#history)
    for line in lines:
        if pattern in line:
            for pline in previous_lines:
                print(lline, end#'')
            print(line, end#'')
            print()
        previous_lines.append(line)

# Example use on a file
if __name__ ## '__main__':
    with open('somefile.txt') as f:
         search(f, 'python', 5)
----

#### Discussion

In [11]:
#如果你已经明确知道了要跳过的元素的个数的话
#那么可以使用itertools.islice()来代替

from itertools import islice

items = list('abc1231015')
for x in islice(items, 3, None): # 指定了获取从第三个到最后的所有元素
#for x in islice(items, None, 3): # 仅仅获取前三个元素
    print(x)

1
2
3
1
0
1
5


In [57]:
#dropwhile和islice,其实就是两个帮助函数，为的就是避免写出下面这种冗余

with open('somefile.txt') as f:
    while True:
        line = next(f, '')
        if not line.startswith('#'):
            break
            
    while True:
        print(line, end='')
        line = next(f, None)
        
#跳过一个可迭代对象的开始部分跟通常的过滤是不同的，比如
#上面代码的第一个部分可能会这样重写

You want to keep a limited history of the last few items seen
during iteration or during some other kind of processing.
#### Solution
Keeping a limited history is a perfect use for a `collections.deque`.
For example, the following code performs a simple text match on a
sequence of lines and prints the matching line along with the previous
N lines of context when found:

[source,python]
----
from collections import deque

def search(lines, pattern, history#5):
    previous_lines # deque(maxlen#history)
    for line in lines:
        if pattern in line:
            for pline in previous_lines:
                print(lline, end#'')
            print(line, end#'')
            print()
        previous_lines.append(line)

# Example use on a file
if __name__ ## '__main__':
    with open('somefile.txt') as f:
         search(f, 'python', 5)
----



#### DiscussionNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneN

NoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNone

NoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNone

NoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNone

...

NoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNone

NoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNone

NoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNone

NoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNone

KeyboardInterrupt: 

In [ ]:
with open('somefile.txt') as f:
    #lines = [line for line in f if not line.startswith('#')] # ?
    lines = ()
    for line in f:
        if not line.startswith('#'):
            lines.append(line)
    for line in lines:
        print(line, end='')
        
#将跳过文件所有注释行，与仅仅跳过开始部分满足测试条件的区别

4.9 排列组合的迭代

问题；你想迭代遍历一个集合中元素的所有可能的排序或组合

解决方案：itertools模块提供了三个函数来解决这个问题

In [20]:
#itertools.permutations它接受一个集合并产生一个元组序列，每个元组
#由集合中所有元素的一个可能排列组成，也就是说通过打乱集合中
#元素排列顺序生成一个元组

items = list('abc')

from itertools import permutations # 排列

for p in permutations(items): 
    print(p)

('a', 'b', 'c')
('a', 'c', 'b')
('b', 'a', 'c')
('b', 'c', 'a')
('c', 'a', 'b')
('c', 'b', 'a')


In [21]:
#如果你想得到指定长度的所有排列，你可以传递一个可选的长度参数

for p in permutations(items,2):
    print(p)

('a', 'b')
('a', 'c')
('b', 'a')
('b', 'c')
('c', 'a')
('c', 'b')


In [22]:
#使用itertools.combinations可得到输入集合中元素的所有的组合

from itertools import combinations # 组合

for c in combinations(items, 3):
    print(c)

('a', 'b', 'c')


In [23]:
for c in combinations(items, 2):
    print(c)

('a', 'b')
('a', 'c')
('b', 'c')


In [24]:
for c in combinations(items, 1):
    print(c)

('a',)
('b',)
('c',)


In [28]:
from itertools import combinations_with_replacement

for c in combinations_with_replacement(items, 3):
    print(c)

#以上仅仅是itertools模块的一部分功能，当碰到看上去复杂的迭代问题
#最后先考虑itertools模块

('a', 'a', 'a')
('a', 'a', 'b')
('a', 'a', 'c')
('a', 'b', 'b')
('a', 'b', 'c')
('a', 'c', 'c')
('b', 'b', 'b')
('b', 'b', 'c')
('b', 'c', 'c')
('c', 'c', 'c')


4.10 序列上索引值迭代

问题：你想在迭代一个序列的同时跟踪正常被处理的元素索引

解决方案：enumerate函数

In [29]:
my_list = list('abc')

for idx, val in enumerate(my_list):
    print(idx, val)

0 a
1 b
2 c


In [30]:
my_list = list('abc')
for idx, val in enumerate(my_list, 1): # 行号从1开始
    print(idx, val)

1 a
2 b
3 c


In [50]:
#这种情况在你遍历文件时想在错误消息中使用行号定位时候非常有用

def parse_data(filename):
    with open(filename, 'rt') as f:
        for lineno, line in enumerate(f, 1): # 行号从1开始
            fields = line.split()
            if len(fields) != 0 :                
                #print(fields)
                try:
                    count = int(fields[1])
                except ValueError as e:
                    print('Line {}: Parse error: {}'.format(lineno, e))
                except IndexError as i:
                    print('Line {}: Index error: {}'.format(lineno,fields[0]))

In [51]:
parse_data('somefile.txt')

Line 1: Parse error: invalid literal for int() with base 10: 'Keeping'
Line 2: Parse error: invalid literal for int() with base 10: 'Problem'
Line 4: Parse error: invalid literal for int() with base 10: '#'
Line 5: Parse error: invalid literal for int() with base 10: 'Problem'
Line 6: Parse error: invalid literal for int() with base 10: 'Problem'
Line 7: Parse error: invalid literal for int() with base 10: 'Problem'
Line 8: Parse error: invalid literal for int() with base 10: 'want'
Line 9: Parse error: invalid literal for int() with base 10: 'iteration'
Line 10: Parse error: invalid literal for int() with base 10: 'Solution'
Line 11: Parse error: invalid literal for int() with base 10: 'a'
Line 12: Parse error: invalid literal for int() with base 10: 'example,'
Line 13: Parse error: invalid literal for int() with base 10: 'of'
Line 14: Parse error: invalid literal for int() with base 10: 'lines'
Line 16: Index error: [source,python]
Line 17: Index error: ----
Line 18: Parse error: inv

In [46]:
#enumerate对于跟踪某些值在列表中出现的位置是很有用的
#如果你想将一个文件中出现的单词映射到它出现的行号上去

from collections import defaultdict

word_summary = defaultdict(list)

with open('somefile.txt', 'r') as f:
    lines = f.readlines() # 每行是列表中的一个元素
    
for idx, line in enumerate(lines):
    words = [w.strip().lower() for w in line.split()]
    for word in words:
        word_summary[word].append(idx)
        
#defaultdict 对于每个单词有一个key，每个key对应的值是一个由单词出现
#的行号组成的列表，如果某个单词在一行中出现过两次
#那么这个行号也会出现两次，同时也作为文本的一个简单统计

In [47]:
print(word_summary)

defaultdict(<class 'list'>, {'###': [0], 'keeping': [0, 10], 'the': [0, 7, 11, 12, 12], 'last': [0, 7], 'n': [0, 13], 'items': [0, 7], '####': [1, 4, 5, 6, 9, 35], 'problem': [1, 4, 5, 6], '#############1111': [2], '############': [3], 'you': [7], 'want': [7], 'to': [7], 'keep': [7], 'a': [7, 10, 10, 10, 11, 11, 29], 'limited': [7, 10], 'history': [7, 10], 'of': [7, 8, 12, 13], 'few': [7], 'seen': [7], 'during': [8, 8], 'iteration': [8], 'or': [8], 'some': [8], 'other': [8], 'kind': [8], 'processing.': [8], 'solution': [9], 'is': [10], 'perfect': [10], 'use': [10, 29], 'for': [10, 11, 21, 23], '`collections.deque`.': [10], 'example,': [11], 'following': [11], 'code': [11], 'performs': [11], 'simple': [11], 'text': [11], 'match': [11], 'on': [11, 29], 'sequence': [12], 'lines': [12, 13], 'and': [12], 'prints': [12], 'matching': [12], 'line': [12, 21], 'along': [12], 'with': [12, 31], 'previous': [12], 'context': [13], 'when': [13], 'found:': [13], '[source,python]': [15], '----': [16, 3

In [50]:
#当你想额外定义一个计数变量的时候，使用enumerate函数会更加简单

#lineno = 1
#for line in f:
#   lineno += 1

#enumerate
#for lineno, line in enumerate(f):

#enumerate函数返回一个enumerate对象实例，它是一个迭代器
#返回连续的包含一个技术和一个值的元组，元组中的值通过传入序列上调用next返回

#陷阱，已经解压后的元组序列上使用enumerate函数

data = [(1, 2), (3, 4), (5, 6), (7, 8)]

for n, (x, y) in enumerate(data):
    pass

for n, x, y in enumerate(data): # error
    pass

ValueError: not enough values to unpack (expected 3, got 2)

4.11 同时迭代多个序列

问题：你想同时迭代多个序列，每次分别从一个序列中取一个元素

解决方案：为了同时迭代多个序列，使用zip函数

In [57]:
xpts = [1, 5, 4, 2, 10, 7]
ypts = [101, 78, 37, 15, 62, 99, 100,101,102]

for x, y in zip(xpts, ypts):
    print(x, y)
    
#迭代长度跟参数中最短序列长度一致

1 101
5 78
4 37
2 15
10 62
7 99


In [60]:
#如果这个不是你想要的结果，那么你还可以使用itertools.zip_longest()函数
from itertools import zip_longest

a = list('abc')
b = list('wxyz')

for i in zip_longest(a, b):
    print(i)
    
for i in zip_longest(a, b, fillvalue=0):
    print(i)

('a', 'w')
('b', 'x')
('c', 'y')
(None, 'z')
('a', 'w')
('b', 'x')
('c', 'y')
(0, 'z')


In [62]:
headers = ['name', 'shares', 'price']
values = ['ACME', 100, 500.1]

#使用zip可以将他们打包并生成一个字典

s = dict(zip(headers, values))

print(s)

{'name': 'ACME', 'shares': 100, 'price': 500.1}


In [64]:
for name, val in zip(headers, values):
    print(name, '=', val)
    
#zip可以接受多于两个的序列的参数，原理同两个序列的一致

name = ACME
shares = 100
price = 500.1


In [65]:
#最后强调的一点就是，zip()创建一个迭代器作为结果返回，如果你需要
#将结对的值存储在列表中，要使用list函数
zip(a, b)

In [66]:
list(zip(a, b))

[('a', 'w'), ('b', 'x'), ('c', 'y')]

4.12 不同集合上元素的迭代

问题：你想在多个对象执行相同的操作，但是这些对象在不同的容器中
，你希望代码在不失可读性的情况下避免重复循环

解决方案：itertools.chain方法可以用来简化这个任务，他接受一个可迭代对象列表
作为输入，并返回一个迭代器，有效的屏蔽掉在多个容器中迭代细节
为了掩饰清楚，考虑下面这个例子

In [1]:
from itertools import chain

a = list('1234')
b = list('xyz')

for x in chain(a, b):
    print(x)

1
2
3
4
x
y
z


In [2]:
#使用chain的一个常见场景是当你想对不同的集合中所有元素执行某些操作的时候

active_items = set()
inactive_items = set()

for item in chain(active_items, inactive_items):
    pass

In [3]:
#这种解决方案要比下面这样使用两个单独的循环更加优雅
for item in active_items:
    pass

for item in inactive_items:
    pass

#itertools.chain接受一个或多个可迭代对象作为输入参数，然后创建一个迭代器
#依次连续的返回每个可迭代对象中的元素，这种方式要比先将序列合并再迭代要高效

#Inefficent
for x in a+b: # 会创建一个全新的序列并要求a和b的类型一致
    pass

for x in chain(a, b): # 不会有这一步，很省内存，并且可迭代对象类型不一样的时候，chain同样很好工作
    pass

4.13 创建数据处理管道

问题：你想以数据管道的方式迭代处理数据，比如，你有个大量的数据需要处理，
但是不能将他们一次性放入内存中

解决方案：生成器函数是一个实现管道机制的好办法，为了掩饰，假定你要处理一个非常
大的日志文件记录

In [9]:
"""
foo/
        access-log-012007.gz
        access-log-022007.gz
        access-log-032007.gz
        .....
        access-log-012000.gz
bar/
        access-log-092007.bz2
        access-log-022008.bz2
        ......
        access-log-072001.bz
        
123.132.5.12 - - [10/Jul/2012:00:18:50 -0500] "GET /robots.txt ...." 200 71
123.132.5.12 - - [10/Jul/2012:00:18:50 -0500] "GET /robots.txt ...." 200 71
123.132.5.12 - - [10/Jul/2012:00:18:50 -0500] "GET /robots.txt ...." 200 71
123.132.5.12 - - [10/Jul/2012:00:18:50 -0500] "GET /robots.txt ...." 200 71
......
"""

'\nfoo/\n        access-log-012007.gz\n        access-log-022007.gz\n        access-log-032007.gz\n        .....\n        access-log-012000.gz\nbar/\n        access-log-092007.bz2\n        access-log-022008.bz2\n        ......\n        access-log-072001.bz\n        \n123.132.5.12 - - [10/Jul/2012:00:18:50 -0500] "GET /robots.txt ...." 200 71\n123.132.5.12 - - [10/Jul/2012:00:18:50 -0500] "GET /robots.txt ...." 200 71\n123.132.5.12 - - [10/Jul/2012:00:18:50 -0500] "GET /robots.txt ...." 200 71\n123.132.5.12 - - [10/Jul/2012:00:18:50 -0500] "GET /robots.txt ...." 200 71\n......\n'

In [8]:
#为了处理这些文件，你可以定义一个由多个执行特定任务独立任务的简单生成器
#函数的容器

import os
import fnmatch # 文件名称的匹配，并且匹配的模式使用的unix shell风格
import gzip
import bz2
import re

def gen_find(filepat, top):
    """
    Parameters:
    ------------
    filepat : file match pattern
                *.jpg
    top : path
            'C:\Program file'
            
    Return:
    --------
    file name with path
    """
    for path, dirlist, filelist in os.walk(top):
        for name in fnmatch.filter(filelist, filepat): # 返回输入列表中，符合特定模式的文件名列表。
            yield os.path.join(path, name)
            
def gen_opener(filenames):
    """
    Return:
    --------
    f : file content
    """
    for filename in filenames:
        if filename.endswith('.gz'):
            f = gzip.open(filename, 'rt')
        elif filename.endswith('.bz2'):
            f = bz2.open(filename, 'rt')
        else:
            f = open(filename, 'rt')
        yield f 
        f.close()
        
def gen_concatenate(iterators):
    """
    Chain a sequence of iterators together into a single sequence
    """
    for it in iterators:
        yield from it
        
def gen_grep(pattern, lines):
    """
    Look for a regex pattern in a sequence of lines
    """
    pat = re.compile(pattern)
    for line in lines:
        if pat.search(line):
            yield line

In [9]:
lognames = gen_find('access-log', 'www')
files = gen_opener(lognames)
lines = gen_concatenate(files)
pylines = gen_grep("(?!)python", lines) 
for line in pylines:
    print(line)

In [15]:
str1 = 'abcdefg acdit'

import re

pata = re.compile(r'cd(?=efg)') # 向字符串右边看，观察后面的字符是否是efg
re.search(pata, str1)

<re.Match object; span=(2, 4), match='cd'>

In [19]:
patb = re.compile(r'cd(?!efg)') # 向右边看，观察后面的字符不是efg的符合条件的
re.search(patb, str1)

<re.Match object; span=(9, 11), match='cd'>

In [36]:
patc = re.compile(r'(?<!a)cd') # 向左边看是否有a字符，如果有，看右边是cd么
re.search(patc, str1)

<re.Match object; span=(2, 4), match='cd'>

In [26]:
patd = re.compile(r'cd(?<!it)') # 向左边看是否cd字符，观察后面的字符不是efg的符合条件的
re.search(patd, str1)

<re.Match object; span=(2, 4), match='cd'>

In [22]:
pate = re.compile(r'cd(?=ef)g') # 环视不消耗字符,逆向环视pattern长度必须是固定的
re.search(pate,str1) # no search

In [23]:
patf = re.compile(r'cd(?=ef)e') # 环视不匹配任何字符，只匹配文本中的特定位置
re.search(patf,str1)

<re.Match object; span=(2, 5), match='cde'>

In [27]:
patg = re.compile(r'cd(?:ef)g')
re.search(patg, str1)

<re.Match object; span=(2, 7), match='cdefg'>

In [28]:
path = re.compile(r'cd(ef)g')
re.search(path, str1)

<re.Match object; span=(2, 7), match='cdefg'>

In [33]:
nums = 3456782345412
patn = re.compile(r'(?<=\d)(?=(\d{3})+$)') # ?
 
m = re.findall(patn, nums) # 千分位

TypeError: expected string or bytes-like object

In [10]:
#如果将来的时候，你想扩展管道，你甚至可以在生成器表达式中包装数据
#下面这个版本计算出传输的字节数并计算其总和

In [11]:
lognames = gen_find('access-log', 'www')
files = gen_opener(lognames)
lines = gen_concatenate(files)
pylines = gen_grep("(?!)python", lines)
bytecolumn = (line.rsplit(None, 1)[1] for line in pylines)
bytes = (int(x) for x in bytecolumn if x != "-")
print("Total", sum(bytes))


Total 0


以管道方式处理数据可以用来解决各类其他问题，包括解析，读取实时数据，定时轮询 等。为了理解上述代码，重点是要明白 yield 语句作为数据的生产者而 for 循环语句作为数 据的消费者。 当这些生成器被连在一起后，每个 yield 会将一个单独的数据元素传递给 迭代处理管道的下一阶段。 在例子最后部分， sum() 函数是最终的程序驱动者，每次从 生成器管道中提取出一个元素这种方式一个非常好的特点是每个生成器函数很小并且都是独立的。这样的话就很容易编 写和维护它们了。 很多时候，这些函数如果比较通用的话可以在其他场景重复使用。 并 且最终将这些组件组合起来的代码看上去非常简单，也很容易理解。 使用这种方式的内存效率也不得不提。上述代码即便是在一个超大型文件目录中也能工作 的很好。 事实上，由于使用了迭代方式处理，代码运行过程中只需要很小很小的内存。 在调用 gen_concatenate() 函数的时候你可能会有些不太明白。 这个函数的目的是将输入 序列拼接成一个很长的行序列。 itertools.chain() 函数同样有类似的功能，但是它需要 将所有可迭代对象最为参数传入。 在上面这个例子中，你可能会写类似这样的语句

lines= itertools.chain(*files) ， 使得 gen_opener() 生成器能被全部消费掉。 但由于 gen_opener() 生成器每次生成一个打开过的文件， 等到下一个迭代步骤时文件就关闭 了，因此 china() 在这里不能这样使用。 上面的方案可以避免这种情况。 gen_concatenate() 函数中出现过 yield from 语句，它将 yield 操作代理到父生成器上 去。 语句 yield from it 简单的返回生成器 it 所产生的所有值。 关于这个我们在4.14小 节会有更进一步的描述。 最后还有一点需要注意的是，管道方式并不是万能的。 有时候你想立即处理所有数据。 然而，即便是这种情况，使用生成器管道也可以将这类问题从逻辑上变为工作流的处理方 式。

4.14 展开嵌套的序列

问题：你想将一个多层嵌套的序列展开成一个单层列表

解决方案：可以写一个包含yield from 语句的递归生成器来轻松解决这个问题

In [7]:
from collections import Iterable

def flatten(items, ignore_types=(str, bytes)):
    for x in items:
        if isinstance(x, Iterable) and not isinstance(x, ignore_types):
            yield from flatten(x) # 返回所有子进程的值，最终返回一个没有嵌套的简单序列了
        else:
            yield x
            
items = [1, 2, [3, 4, [5, 6], 7], 8]
for x in flatten(items):
    print(x)

1
2
3
4
5
6
7
8


d:\program files (x86)\python37\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  """Entry point for launching an IPython kernel.


In [8]:
items = ['Dave', 'Paula', ['Thomas', 'Lewis']]
for x in flatten(items):
    print(x)

Dave
Paula
Thomas
Lewis


In [23]:
#yield from 在你想在生成器中调用其他生成器作为子例程的时候非常有用
#如果你不使用它的话，那么必须写额外的for循环了

def flatten(items, ignore_types=(str,bytes)):
    for x in items:
        if isinstance(x, Iterable) and not isinstance(x, ignore_types):
            for i in flatten(x):
                yield i
        else:
            yield x

4.15 顺序迭代合并后的排序迭代对象

问题：你有一系列排序序列，想将他们合并后得到一个排序序列并在上面迭代遍历

解决方案：heapq.merge()函数可以帮你解决这个问题

In [3]:
import heapq

a = [111, 41, 17, 10]
b = [2, 5, 6, 11]
for c in heapq.merge(a, b):
    print(c)

2
5
6
11
111
41
17
10


In [6]:
a = [111, 41, 17, 10]
b = [21, 5, 611, 9]
for c in heapq.merge(a, b):
    print(c)
# 序列比较，谁小打印谁

21
5
111
41
17
10
611
9


In [2]:
#heapq.merge可迭代特性意味着他不会立马读取所有序列，这就意味着
#你可以在非常长的序列使用它，而不会太大开销

with open('sorted_file_1','rt') as file1, open('sorted_file_2','rt') as file2, open('merged_file', 'wt') as outf:
    for line in heapq.merge(file1, file2):
        outf.write(line)

#heapq.merge需要所有输入序列必须是排过序的，特别的
#它并不会预先读取数据到堆栈中或者预先排序，也不会对输入做任何的排序检测
#它仅仅是检查所有序列的开始部分并返回最小的那个，这个过程一直持续到所有输入序列中
#元素都被遍历完

FileNotFoundError: [Errno 2] No such file or directory: 'sorted_file_1'

4.16 迭代器代替while无限循环

问题：使用while循环来迭代处理数据，因为他需要调用某个函数或者一般迭代模式不同的测试条件，能不能采用迭代器重写这个循环

In [1]:
#解决方案

chunksize = 8192

def reader(s):
    while True:
        data = s.recv(chunksize) # 多用于网络编程，当与socket断开连接时，会收到空字符串
        if data == b' ':
            break
        process_data(data)
        
#这种代码通常可以使用iter来代替

def reader2(s):
    for chunk in iter(lambda : s.recv(chunksize), b''): # 这个匿名函数没理解
        pass

#如果你怀疑他到底能不能正常工作，可以试验一下简单的例子
import sys

f = open('somefile.txt')
for chunk in iter(lambda: f.read(10), ''):
    n = sys.stdout.write(chunk) # print = sys.stdout.write(obj + \n)


### Keeping the Last N Items
#### Problem
#############1111
############
#### Problem
#### Problem
#### Problem
You want to keep a limited history of the last few items seen
during iteration or during some other kind of processing.
#### Solution
Keeping a limited history is a perfect use for a `collections.deque`.
For example, the following code performs a simple text match on a
sequence of lines and prints the matching line along with the previous
N lines of context when found:

[source,python]
----
from collections import deque

def search(lines, pattern, history#5):
    previous_lines # deque(maxlen#history)
    for line in lines:
        if pattern in line:
            for pline in previous_lines:
                print(lline, end#'')
            print(line, end#'')
            print()
        previous_lines.append(line)

# Example use on a file
if __name__ ## '__main__':
    with open('somefile.txt') as f:
         search(f, 'python', 5)
----

#### Discussion

iter 函数一个鲜为人知的特性是它接受一个可选的 callable 对象和一个标记(结尾)值作 为输入参数。 当以这种方式使用的时候，它会创建一个迭代器， 这个迭代器会不断调用 callable 对象直到返回值和标记值相等为止。 这种特殊的方法对于一些特定的会被重复调用的函数很有效果，比如涉及到I/O调用的函数。 

举例来讲，如果你想从套接字或文件中以数据块的方式读取数据，通常你得要不断 重复的执行 read() 或 recv() ， 并在后面紧跟一个文件结尾测试来决定是否终止。这节中的方案使用一个简单的 iter() 调用就可以将两者结合起来了。 其中 lambda 函数参数是为了创建一个无参的 callable 对象，并为recv 或read() 方法提供了size 参数